In [4]:
!pip install python-dotenv psycopg2-binary

     |████████████████████████████████| 3.0 MB 50 kB/s  eta 0:00:01     |██████████████████              | 1.7 MB 2.8 MB/s eta 0:00:01


In [4]:
import pandas as pd
import yaml
import os
from sqlalchemy import create_engine

# Load configuração do YAML
with open('tables.yml', 'r') as file:
    config = yaml.safe_load(file)

# Cria pasta 'data' se não existir
os.makedirs("data", exist_ok=True)

# Configurações de conexão com o PostgreSQL
USER = "postgres"
PASSWORD = "admin"
HOST = "192.168.0.202"
PORT = "5432"
DB = "postgres"

# Cria engine SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")

def extrair_tabela_pequena(source, target):
    print(f"Extraindo tabela pequena: {source}")
    df = pd.read_sql_query(f"SELECT * FROM {source}", engine)
    output_path = f"data/{target}.csv"
    df.to_csv(output_path, index=False)
    print(f"✅ Salvou: {output_path} ({len(df)} linhas)")

def extrair_tabela_grande(source, target, partition_config):
    col = partition_config["column"]
    lower = partition_config["lower_bound"]
    upper = partition_config["upper_bound"]
    num_parts = partition_config["num_partitions"]

    step = (upper - lower + 1) // num_parts

    for i in range(num_parts):
        start = lower + i * step
        end = start + step - 1 if i < num_parts - 1 else upper

        print(f"Extraindo {target}_{i+1:02d}: {col} de {start} até {end}")

        query = f"""
            SELECT * FROM {source}
            WHERE {col} BETWEEN %s AND %s
        """
        df = pd.read_sql_query(query, engine, params=(start, end))
        output_path = f"data/{target}_{i+1:02d}.csv"
        df.to_csv(output_path, index=False)
        print(f"✅ Salvou: {output_path} ({len(df)} linhas)")

# Loop pelas tabelas do YAML
for table in config['folders']:
    source = table['source']
    target = table['target']
    large = table.get('large_table', False)

    if large:
        extrair_tabela_grande(source, target, table['partition_config'])
    else:
        extrair_tabela_pequena(source, target)

print("✅ Extração finalizada.")


Extraindo tabela pequena: clientes
✅ Salvou: data/clientes.csv (1 linhas)
✅ Extração finalizada.


In [6]:
import pandas as pd
import yaml
import os
from sqlalchemy import create_engine

# Load configuração do YAML
with open('tables.yml', 'r') as file:
    config = yaml.safe_load(file)

# Cria pasta 'data' se não existir
os.makedirs("data", exist_ok=True)

# Configurações de conexão com o PostgreSQL
USER = "postgres"
PASSWORD = "admin"
HOST = "192.168.0.202"
PORT = "5432"
DB = "postgres"

# Cria engine SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")

def extrair_tabela_pequena(source, target):
    print(f"Extraindo tabela pequena: {source}")
    df = pd.read_sql_query(f"SELECT * FROM {source}", engine)
    output_path = f"data/{target}.csv"
    df.to_csv(output_path, index=False)
    print(f"✅ Salvou: {output_path} ({len(df)} linhas)")

def extrair_tabela_grande(source, target, table_name, partition_config):
    col = partition_config["column"]
    lower = partition_config["lower_bound"]
    upper = partition_config["upper_bound"]
    num_parts = partition_config["num_partitions"]

    step = (upper - lower + 1) // num_parts

    for i in range(num_parts):
        start = lower + i * step
        end = start + step - 1 if i < num_parts - 1 else upper

        print(f"Extraindo {target}_{i+1:02d}: {col} de {start} até {end}")

        query = f"""
            WITH tb_auditoria AS (
                SELECT ID_REGISTRO, NOME_TABELA, data,
                       ROW_NUMBER() OVER (PARTITION BY ID_REGISTRO ORDER BY DATA DESC) AS RN
                FROM PUBLIC.AUDITORIA
                WHERE 1=1
                    AND NOME_TABELA = %s
                    AND data >= CURRENT_DATE - 1
                    AND data <= CURRENT_DATE
                    AND ID_REGISTRO BETWEEN %s AND %s
            )
            SELECT c.*
            FROM tb_auditoria audit
            INNER JOIN {source} c ON c.id = audit.ID_REGISTRO
            WHERE audit.RN = 1
        """
        df = pd.read_sql_query(query, engine, params=(table_name, start, end))
        output_path = f"data/{target}_{i+1:02d}.csv"
        df.to_csv(output_path, index=False)
        print(f"✅ Salvou: {output_path} ({len(df)} linhas)")

# Loop pelas tabelas do YAML
for table in config['folders']:
    source = table['source']
    target = table['target']
    large = table.get('large_table', False)

    if large:
        table_name = table.get('table_name', target)  # Usa 'target' como fallback se 'table_name' não estiver definido
        extrair_tabela_grande(source, target, table_name, table['partition_config'])
    else:
        extrair_tabela_pequena(source, target)

print("✅ Extração finalizada.")

Extraindo clientes_01: id de 1 até 100
✅ Salvou: data/clientes_01.csv (1 linhas)
Extraindo clientes_02: id de 101 até 200
✅ Salvou: data/clientes_02.csv (0 linhas)
Extraindo clientes_03: id de 201 até 300
✅ Salvou: data/clientes_03.csv (0 linhas)
Extraindo clientes_04: id de 301 até 400
✅ Salvou: data/clientes_04.csv (0 linhas)
Extraindo clientes_05: id de 401 até 500
✅ Salvou: data/clientes_05.csv (0 linhas)
Extraindo clientes_06: id de 501 até 600
✅ Salvou: data/clientes_06.csv (0 linhas)
Extraindo clientes_07: id de 601 até 700
✅ Salvou: data/clientes_07.csv (0 linhas)
Extraindo clientes_08: id de 701 até 800
✅ Salvou: data/clientes_08.csv (0 linhas)
Extraindo clientes_09: id de 801 até 900
✅ Salvou: data/clientes_09.csv (0 linhas)
Extraindo clientes_10: id de 901 até 1000
✅ Salvou: data/clientes_10.csv (0 linhas)
✅ Extração finalizada.


In [1]:
import pandas as pd
import yaml
import os
from sqlalchemy import create_engine

# Load configuração do YAML
with open('tables.yml', 'r') as file:
    config = yaml.safe_load(file)

# Cria pasta 'data' se não existir
os.makedirs("data", exist_ok=True)

# Configurações de conexão com o PostgreSQL
USER = "postgres"
PASSWORD = "admin"
HOST = "192.168.0.202"
PORT = "5432"
DB = "postgres"

# Cria engine SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")

def extrair_tabela_pequena(source, target, table_name):
    print(f"Extraindo tabela pequena: {source}")
    query = f"""
        WITH tb_auditoria AS (
            SELECT ID_REGISTRO, NOME_TABELA, data,
                   ROW_NUMBER() OVER (PARTITION BY ID_REGISTRO ORDER BY DATA DESC) AS RN
            FROM PUBLIC.AUDITORIA
            WHERE 1=1
                AND NOME_TABELA = %s
                AND data >= CURRENT_DATE - 1
                AND data <= CURRENT_DATE
        )
        SELECT c.*
        FROM tb_auditoria audit
        INNER JOIN {source} c ON c.id = audit.ID_REGISTRO
        WHERE audit.RN = 1
    """
    df = pd.read_sql_query(query, engine, params=(table_name,))
    output_path = f"data/{target}.csv"
    df.to_csv(output_path, index=False)
    print(f"✅ Salvou: {output_path} ({len(df)} linhas)")

def extrair_tabela_grande(source, target, table_name, partition_config):
    col = partition_config["column"]
    lower = partition_config["lower_bound"]
    upper = partition_config["upper_bound"]
    num_parts = partition_config["num_partitions"]

    step = (upper - lower + 1) // num_parts

    for i in range(num_parts):
        start = lower + i * step
        end = start + step - 1 if i < num_parts - 1 else upper

        print(f"Extraindo {target}_{i+1:02d}: {col} de {start} até {end}")

        query = f"""
            WITH tb_auditoria AS (
                SELECT ID_REGISTRO, NOME_TABELA, data,
                       ROW_NUMBER() OVER (PARTITION BY ID_REGISTRO ORDER BY DATA DESC) AS RN
                FROM PUBLIC.AUDITORIA
                WHERE 1=1
                    AND NOME_TABELA = %s
                    AND data >= CURRENT_DATE - 1
                    AND data <= CURRENT_DATE
                    AND ID_REGISTRO BETWEEN %s AND %s
            )
            SELECT c.*
            FROM tb_auditoria audit
            INNER JOIN {source} c ON c.id = audit.ID_REGISTRO
            WHERE audit.RN = 1
        """
        df = pd.read_sql_query(query, engine, params=(table_name, start, end))
        output_path = f"data/{target}_{i+1:02d}.csv"
        df.to_csv(output_path, index=False)
        print(f"✅ Salvou: {output_path} ({len(df)} linhas)")

# Loop pelas tabelas do YAML
for table in config['folders']:
    source = table['source']
    target = table['target']
    table_name = table.get('table_name', target)  # Usa 'target' como fallback
    large = table.get('large_table', False)

    if large:
        extrair_tabela_grande(source, target, table_name, table['partition_config'])
    else:
        extrair_tabela_pequena(source, target, table_name)

print("✅ Extração finalizada.")

Extraindo clientes_01: id de 1 até 100
✅ Salvou: data/clientes_01.csv (98 linhas)
Extraindo clientes_02: id de 101 até 200
✅ Salvou: data/clientes_02.csv (100 linhas)
Extraindo clientes_03: id de 201 até 300
✅ Salvou: data/clientes_03.csv (100 linhas)
Extraindo clientes_04: id de 301 até 400
✅ Salvou: data/clientes_04.csv (100 linhas)
Extraindo clientes_05: id de 401 até 500
✅ Salvou: data/clientes_05.csv (100 linhas)
Extraindo clientes_06: id de 501 até 600
✅ Salvou: data/clientes_06.csv (100 linhas)
Extraindo clientes_07: id de 601 até 700
✅ Salvou: data/clientes_07.csv (100 linhas)
Extraindo clientes_08: id de 701 até 800
✅ Salvou: data/clientes_08.csv (100 linhas)
Extraindo clientes_09: id de 801 até 900
✅ Salvou: data/clientes_09.csv (100 linhas)
Extraindo clientes_10: id de 901 até 1000
✅ Salvou: data/clientes_10.csv (100 linhas)
✅ Extração finalizada.


In [1]:
import pandas as pd
import yaml
import os
from sqlalchemy import create_engine
import logging
import numpy as np

# Configuração do Logging
logging.basicConfig(
    level=logging.INFO,
    format='{"level": "%(levelname)s", "message": "%(message)s"}',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("etl_postgres")

# Load configuração do YAML
with open('tables.yml', 'r') as file:
    config = yaml.safe_load(file)

# Cria pasta 'data' se não existir
os.makedirs("data", exist_ok=True)

# Configurações de conexão com o PostgreSQL
USER = "postgres"
PASSWORD = "admin"
HOST = "192.168.0.202"
PORT = "5432"
DB = "postgres"

# Cria engine SQLAlchemy
engine = create_engine(
    f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}",
    connect_args={'connect_timeout': 300}
)

def get_id_bounds(table_name, date_filter_days=1):
    query = """
        SELECT MIN(ID_REGISTRO), MAX(ID_REGISTRO)
        FROM PUBLIC.AUDITORIA
        WHERE NOME_TABELA = %s
          AND data >= CURRENT_DATE - %s
          AND data <= CURRENT_DATE
    """
    try:
        result = pd.read_sql_query(query, engine, params=(table_name, date_filter_days))
        min_id = result.iloc[0]['min']
        max_id = result.iloc[0]['max']
        # Converter numpy.int64 para int Python
        min_id = int(min_id) if min_id is not None else 1
        max_id = int(max_id) if max_id is not None else 1
        return min_id, max_id
    except Exception as e:
        logger.error(f"❌ Erro ao obter limites de ID para {table_name}: {str(e)}")
        return 1, 1

def contar_registros_esperados(source, table_name, date_filter_days=1, lower=None, upper=None):
    query = """
        WITH tb_auditoria AS (
            SELECT ID_REGISTRO, NOME_TABELA, data,
                   ROW_NUMBER() OVER (PARTITION BY ID_REGISTRO ORDER BY DATA DESC) AS RN
            FROM PUBLIC.AUDITORIA
            WHERE 1=1
                AND NOME_TABELA = %s
                AND data >= CURRENT_DATE - %s
                AND data <= CURRENT_DATE
    """
    params = [table_name, date_filter_days]
    if lower is not None and upper is not None:
        query += " AND ID_REGISTRO BETWEEN %s AND %s"
        # Garantir que lower e upper sejam int
        params.extend([int(lower), int(upper)])
    query += """
        )
        SELECT COUNT(*)
        FROM tb_auditoria audit
        INNER JOIN {} c ON c.id = audit.ID_REGISTRO
        WHERE audit.RN = 1
    """.format(source)
    
    try:
        count = pd.read_sql_query(query, engine, params=tuple(params))['count'].iloc[0]
        return count
    except Exception as e:
        logger.error(f"❌ Erro ao contar registros para {source}: {str(e)}")
        return 0

def extrair_tabela_pequena(source, target, table_name, date_filter_days=1):
    logger.info(f"Extraindo tabela pequena: {source}")
    
    # Contar registros esperados
    expected_rows = contar_registros_esperados(source, table_name, date_filter_days)
    logger.info(f"Esperado {expected_rows} linhas para {source}")

    query = """
        WITH tb_auditoria AS (
            SELECT ID_REGISTRO, NOME_TABELA, data,
                   ROW_NUMBER() OVER (PARTITION BY ID_REGISTRO ORDER BY DATA DESC) AS RN
            FROM PUBLIC.AUDITORIA
            WHERE 1=1
                AND NOME_TABELA = %s
                AND data >= CURRENT_DATE - %s
                AND data <= CURRENT_DATE
        )
        SELECT c.*
        FROM tb_auditoria audit
        INNER JOIN {} c ON c.id = audit.ID_REGISTRO
        WHERE audit.RN = 1
    """.format(source)
    
    output_path = f"data/{target}.csv"
    total_rows = 0
    first_chunk = True

    try:
        for chunk in pd.read_sql_query(query, engine, params=(table_name, date_filter_days), chunksize=100000):
            mode = 'w' if first_chunk else 'a'
            header = first_chunk
            chunk.to_csv(output_path, mode=mode, header=header, index=False)
            total_rows += len(chunk)
            logger.info(f"Processado chunk de {len(chunk)} linhas para {output_path}")
            first_chunk = False
    except Exception as e:
        logger.error(f"❌ Erro ao processar tabela pequena {source}: {str(e)}")
        return

    logger.info(f"✅ Salvou: {output_path} ({total_rows} linhas)")
    if total_rows != expected_rows:
        logger.warning(f"⚠️ Discrepância: Extraído {total_rows} linhas, esperado {expected_rows}")

def extrair_tabela_grande(source, target, table_name, partition_config):
    col = partition_config["column"]
    num_parts = partition_config.get("num_partitions", 100)  # Padrão: 100 partições
    date_filter_days = partition_config.get("date_filter_days", 1)  # Padrão: 1 dia

    # Obter limites dinâmicos de ID_REGISTRO
    lower, upper = get_id_bounds(table_name, date_filter_days)
    logger.info(f"Limites de {col} para {table_name}: {lower} até {upper}")

    # Contar registros esperados
    expected_rows = contar_registros_esperados(source, table_name, date_filter_days, lower, upper)
    logger.info(f"Esperado {expected_rows} linhas para {source} ({col} de {lower} até {upper})")

    # Ajustar número de partições se o intervalo for pequeno
    if upper - lower + 1 < num_parts:
        num_parts = max(1, int(upper - lower + 1))
    step = (upper - lower + 1) // num_parts
    logger.info(f"Particionando {target}: {col} de {lower} até {upper} em {num_parts} partições")

    total_rows_all = 0
    for i in range(num_parts):
        start = lower + i * step
        end = start + step - 1 if i < num_parts - 1 else upper

        logger.info(f"Extraindo {target}_{i+1:02d}: {col} de {start} até {end}")

        query = """
            WITH tb_auditoria AS (
                SELECT ID_REGISTRO, NOME_TABELA, data,
                       ROW_NUMBER() OVER (PARTITION BY ID_REGISTRO ORDER BY DATA DESC) AS RN
                FROM PUBLIC.AUDITORIA
                WHERE 1=1
                    AND NOME_TABELA = %s
                    AND data >= CURRENT_DATE - %s
                    AND data <= CURRENT_DATE
                    AND ID_REGISTRO BETWEEN %s AND %s
            )
            SELECT c.*
            FROM tb_auditoria audit
            INNER JOIN {} c ON c.id = audit.ID_REGISTRO
            WHERE audit.RN = 1
        """.format(source)

        output_path = f"data/{target}_{i+1:02d}.csv"
        total_rows = 0
        first_chunk = True

        try:
            for chunk in pd.read_sql_query(query, engine, params=(table_name, date_filter_days, int(start), int(end)), chunksize=100000):
                mode = 'w' if first_chunk else 'a'
                header = first_chunk
                chunk.to_csv(output_path, mode=mode, header=header, index=False)
                total_rows += len(chunk)
                logger.info(f"Processado chunk de {len(chunk)} linhas para {output_path}")
                first_chunk = False
            total_rows_all += total_rows
        except Exception as e:
            logger.error(f"❌ Erro ao processar partição {target}_{i+1:02d}: {str(e)}")
            continue

        logger.info(f"✅ Salvou: {output_path} ({total_rows} linhas)")

    logger.info(f"Total extraído para {target}: {total_rows_all} linhas")
    if total_rows_all != expected_rows:
        logger.warning(f"⚠️ Discrepância: Extraído {total_rows_all} linhas, esperado {expected_rows}")

# Loop pelas tabelas do YAML
for table in config['folders']:
    source = table['source']
    target = table['target']
    table_name = table.get('table_name', target)  # Usa 'target' como fallback
    large = table.get('large_table', False)

    try:
        if large:
            extrair_tabela_grande(source, target, table_name, table['partition_config'])
        else:
            date_filter_days = table.get('date_filter_days', 1)
            extrair_tabela_pequena(source, target, table_name, date_filter_days)
    except Exception as e:
        logger.error(f"❌ Erro ao processar tabela {source}: {str(e)}")
        continue

logger.info("✅ Extração finalizada.")

{"level": "INFO", "message": "Limites de id para clientes: 1 até 6110003"}
{"level": "INFO", "message": "Esperado 100000 linhas para public.clientes (id de 1 até 6110003)"}
{"level": "INFO", "message": "Particionando clientes: id de 1 até 6110003 em 20 partições"}
{"level": "INFO", "message": "Extraindo clientes_01: id de 1 até 305500"}
{"level": "INFO", "message": "Processado chunk de 0 linhas para data/clientes_01.csv"}
{"level": "INFO", "message": "✅ Salvou: data/clientes_01.csv (0 linhas)"}
{"level": "INFO", "message": "Extraindo clientes_02: id de 305501 até 611000"}
{"level": "INFO", "message": "Processado chunk de 0 linhas para data/clientes_02.csv"}
{"level": "INFO", "message": "✅ Salvou: data/clientes_02.csv (0 linhas)"}
{"level": "INFO", "message": "Extraindo clientes_03: id de 611001 até 916500"}
{"level": "INFO", "message": "Processado chunk de 0 linhas para data/clientes_03.csv"}
{"level": "INFO", "message": "✅ Salvou: data/clientes_03.csv (0 linhas)"}
{"level": "INFO", "m